## Environment Setup

In [ ]:
!cd LLM-Neo && pip install -e ".[torch,metrics]"
!pip install deepspeed

In [ ]:
import os

# Inject environment flag so DeepSpeed won't complain
os.environ["FORCE_TORCHRUN"] = "1"

In [ ]:
!huggingface-cli login --token <hf token>

In [ ]:
# if running on terminal:
# export FORCE_TORCHRUN=1

## LLM-NEO
To change student or teacher model, modify arguments in ```script_Neo.py```

In [ ]:
!cd LLM-Neo && python3 script_Neo.py

In [ ]:
!cd LLM-Neo && bash run_train.sh 
# alternatively you can train in a terminal environment

## Save model
It is recommended to create a new folder for the final model with all the relevant files. The path for the model folder should ideally be:
```/LLM-Neo/saves/llama3-3b/lora/neo/final_model/```

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

# Step 1: Load PEFT config
peft_model_id = "/LLM-Neo/saves/llama3-3b/lora/neo/final_model/" # update path
peft_config = PeftConfig.from_pretrained(peft_model_id)

# Step 2: Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    device_map="auto",
    torch_dtype="auto"
)

# Step 3: Load PEFT adapter
model = PeftModel.from_pretrained(base_model, peft_model_id)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)

In [ ]:
model

In [ ]:
# Optional: Merge PEFT adapter with base model (useful for inference)
model = model.merge_and_unload()

In [ ]:
repo_name = "llama_3b_sparse_neo" # Choose a name for your repository
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)